In [1]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

In [2]:
data = Dataset.load_builtin('ml-100k')

trainset, testset = train_test_split(data, test_size=.25, random_state=0)

In [3]:
algo = SVD()
algo.fit(trainset)

In [4]:
predictions = algo.test(testset)
print('prediction type:', type(predictions), 'size:', len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

prediction type: <class 'list'> size: 25000
prediction 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.4537080947718937, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.878352710987769, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=3.8826265786262195, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.559433148953686, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.5460576354235234, details={'was_impossible': False})]

In [5]:
[(pred.uid, pred.iid, pred.est) for pred in predictions[:3]]

[('120', '282', 3.4537080947718937),
 ('882', '291', 3.878352710987769),
 ('535', '507', 3.8826265786262195)]

In [6]:
uid = str(196)
iid = str(302)
pred = algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.03   {'was_impossible': False}


In [7]:
accuracy.rmse(predictions)

RMSE: 0.9508


0.9507989712690431

In [8]:
import pandas as pd

ratings = pd.read_csv('./train_data/ml-latest-small/ratings.csv')
ratings.to_csv('./train_data/ml-latest-small/ratings_noh.csv', index=False, header=False)

In [9]:
from surprise import Reader

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
data = Dataset.load_from_file('./train_data/ml-latest-small/ratings_noh.csv', reader=reader)

In [10]:
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

In [11]:
import pandas as pd
from surprise import Reader, Dataset

ratings = pd.read_csv('./train_data/ml-latest-small/ratings.csv')
reader = Reader(rating_scale=(0.5, 5.0))

data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

In [12]:
from surprise.model_selection import cross_validate

ratings = pd.read_csv('./train_data/ml-latest-small/ratings.csv')
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

algo = SVD(random_state=0)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8691  0.8791  0.8715  0.8792  0.8756  0.8749  0.0040  
MAE (testset)     0.6665  0.6766  0.6710  0.6746  0.6722  0.6722  0.0034  
Fit time          3.23    3.26    3.25    3.30    3.24    3.26    0.02    
Test time         0.10    0.15    0.14    0.10    0.10    0.12    0.02    


{'test_rmse': array([0.86912521, 0.87906617, 0.87150908, 0.87916024, 0.87564908]),
 'test_mae': array([0.66652584, 0.67655899, 0.67102389, 0.67458597, 0.672158  ]),
 'fit_time': (3.2317893505096436,
  3.255190372467041,
  3.251823902130127,
  3.295401096343994,
  3.243802309036255),
 'test_time': (0.09702229499816895,
  0.1470344066619873,
  0.1420764923095703,
  0.09602165222167969,
  0.09502196311950684)}

In [13]:
from surprise.model_selection import GridSearchCV

param_grid = {'n_epochs': [20, 40, 60], 'n_factors': [50, 100, 200]}

gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3, n_jobs=-1)
gs.fit(data)
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8782731068588391
{'n_epochs': 20, 'n_factors': 50}


In [15]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
data_folds = DatasetAutoFolds(ratings_file='./train_data/ml-latest-small/ratings_noh.csv', reader=reader)

trainset = data_folds.build_full_trainset()

In [16]:
algo = SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)

In [18]:
movies = pd.read_csv('./train_data/ml-latest-small/movies.csv')

movieIds = ratings[ratings['userId'] == 9]['movieId']
if movieIds[movieIds == 42].count() == 0:
    print('사용자 아이디 9는 영화 아이디 42의 평점 없음')

print(movies[movies['movieId'] == 42])

사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [19]:
uid = str(9)
iid = str(42)

pred = algo.predict(uid, iid, verbose=True)

user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}


In [20]:
def get_unseen_surprise(ratings, movies, userId):
    seen_movies = ratings[ratings['userId'] == userId]['movieId'].tolist()
    total_movies = movies['movieId'].tolist()
    
    unseen_movies = [movie for movie in total_movies if movie not in seen_movies]
    print('평점 매긴 영화 수:', len(seen_movies), '추천 대상 영화 수:', len(unseen_movies), '전체 영화 수:', len(total_movies))
    return unseen_movies

In [21]:
unseen_movies = get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화 수: 46 추천 대상 영화 수: 9696 전체 영화 수: 9742


In [24]:
from operator import attrgetter
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):
    predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]
    predictions.sort(key=attrgetter('est'), reverse=True)
    top_predctions = predictions[:top_n]
    
    top_movie_ids = [int(pred.iid) for pred in top_predctions]
    top_movie_rating = [pred.est for pred in top_predctions]
    top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']
    
    top_movie_preds = [(iid, title, rating) for iid, title, rating in zip(top_movie_ids, top_movie_titles, top_movie_rating)]
    
    return top_movie_preds

In [25]:
unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)

print('#### Top-10 추천 영화 리스트 ####')
for top_movie in top_movie_preds:
    print(top_movie[1], ":", top_movie[2])

평점 매긴 영화 수: 46 추천 대상 영화 수: 9696 전체 영화 수: 9742
#### Top-10 추천 영화 리스트 ####
Usual Suspects, The (1995) : 4.306302135700814
Star Wars: Episode IV - A New Hope (1977) : 4.281663842987387
Pulp Fiction (1994) : 4.278152632122759
Silence of the Lambs, The (1991) : 4.226073566460876
Godfather, The (1972) : 4.1918097904381995
Streetcar Named Desire, A (1951) : 4.154746591122658
Star Wars: Episode V - The Empire Strikes Back (1980) : 4.122016128534504
Star Wars: Episode VI - Return of the Jedi (1983) : 4.108009609093436
Goodfellas (1990) : 4.083464936588478
Glory (1989) : 4.07887165526957
